## Importar librerias necesarias

In [1]:
from abc import ABC, abstractmethod
import numpy as np
from os import urandom
import random
import pandas as pd
import itertools
from math import comb
from math import sqrt, ceil
from typing import List, Tuple, Optional
from collections import defaultdict

## Manejo de excepciones en las clases de cifrado

In [2]:
class NotImplementedException(Exception):
    def __init__(self, func, cls):
        self.func = func
        self.cls = cls
        super().__init__(f"Function {func} (of class {cls}) is not implemented!")

## Clase de Abstract cipher

In [3]:
class AbstractCipher(ABC):
    """ Abstract cipher class containing all methods a cipher class should implement """

    """ Data types for all the supported word sizes """
    DTYPES = {
        2: np.uint8,
        4: np.uint8,
        8: np.uint8,
        16: np.uint16,
        32: np.uint32
    }

    def __init__(
            self, n_rounds, word_size, n_words, n_main_key_words, n_round_key_words,
            use_key_schedule=True, main_key_word_size=None, round_key_word_size=None
    ):
        """
        Initializes a cipher object
        :param n_rounds: The number of rounds used for de-/encryption
        :param word_size: The size (in bits) of a ciphertext word
        :param n_words: The number of words of one ciphertext
        :param n_main_key_words: The number of words in the main key
        :param n_round_key_words: The number of words in each round key
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        :param main_key_word_size: The size (in bits) of a main key word ('None' means the same as word_size)
        :param round_key_word_size: The size (in bits) of a round key word ('None' means the same as word_size)
        """
        self.n_rounds = n_rounds
        self.word_size = word_size
        self.word_dtype = self.DTYPES.get(self.word_size, None)
        if self.word_dtype is None:
            raise Exception(f'Error: Unexpected word size {self.word_size}')
        self.mask_val = 2 ** self.word_size - 1
        self.n_words = n_words
        self.n_main_key_words = n_main_key_words
        self.n_round_key_words = n_round_key_words
        self.use_key_schedule = use_key_schedule
        self.main_key_word_size = main_key_word_size if main_key_word_size is not None else word_size
        self.main_key_word_dtype = self.DTYPES.get(self.main_key_word_size, None)
        if self.main_key_word_dtype is None:
            raise Exception(f'Error: Unexpected word size {self.main_key_word_size}')
        self.round_key_word_size = round_key_word_size if round_key_word_size is not None else word_size
        self.round_key_word_dtype = self.DTYPES.get(self.round_key_word_size, None)
        if self.round_key_word_dtype is None:
            raise Exception(f'Error: Unexpected word size {self.round_key_word_size}')

    def get_word_size(self):
        """
        :return: The size (in bits) of one word (which could be the size of an s-box or of the right/left side)
        """
        return self.word_size

    def get_n_words(self):
        """
        :return: The number of words in one ciphertext
        """
        return self.n_words

    def get_block_size(self):
        """
        :return: The size (in bits) of one ciphertext
        """
        return self.word_size * self.n_words

    def get_n_rounds(self):
        """
        :return: The number of rounds
        """
        return self.n_rounds

    def set_n_rounds(self, new_n_rounds):
        """
        Sets the number of rounds
        :param new_n_rounds: The new number of rounds
        """
        self.n_rounds = new_n_rounds

    @staticmethod
    def bytes_per_word(word_size):
        """
        :param word_size: The word size (in bits)
        :return: Returns the number of bytes to represent a word of word_size bits
        """
        return word_size // 8 + (1 if (word_size % 8) else 0)

    @abstractmethod
    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        pass

    def encrypt(self, p, keys):
        """
        Encrypt by applying the round function for each given round key
        :param p: The plaintext
        :param keys: A list of round keys
        :return: The encryption of p under the round keys in keys
        """
        state = p
        for i in range(len(keys)):
            state = self.encrypt_one_round(state, keys[i], self.get_rc(i))
        return state

    @abstractmethod
    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        pass

    def decrypt(self, c, keys):
        """
        Decrypt by applying the inverse round function for each given key
        :param c: The ciphertext
        :param keys: A list of round keys
        :return: The decryption of c under the round keys in keys
        """
        state = c
        for i in range(len(keys) - 1, -1, -1):
            state = self.decrypt_one_round(state, keys[i], self.get_rc(i))
        return state

    @abstractmethod
    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        pass

    def get_rc(self, r):
        """
        :param r: The round
        :return: The round constant for round r
        """
        return None

    def draw_keys(self, n_samples):
        """
        :param n_samples: How many keys to draw
        :return: An array of keys
        """
        if self.use_key_schedule:
            bytes_per_word = self.bytes_per_word(self.main_key_word_size)
            main_key = np.frombuffer(
                urandom(self.n_main_key_words * bytes_per_word * n_samples), dtype=self.main_key_word_dtype
            ).reshape(self.n_main_key_words, n_samples)
            if self.main_key_word_size < 8:
                # Note: If the word size is greater than 8, it will always fit the dtype for the ciphers we use
                main_key = np.right_shift(main_key, 8 - self.main_key_word_size)
            return self.key_schedule(main_key)
        else:
            bytes_per_word = self.bytes_per_word(self.round_key_word_size)
            round_keys = np.frombuffer(
                urandom(self.n_rounds * self.n_round_key_words * bytes_per_word * n_samples),
                dtype=self.round_key_word_dtype
            ).reshape(self.n_rounds, self.n_round_key_words, n_samples)
            if self.round_key_word_size < 8:
                # Note: If the word size is greater than 8, it will always fit the dtype for the ciphers we use
                round_keys = np.right_shift(round_keys, 8 - self.round_key_word_size)
            return round_keys

    def draw_plaintexts(self, n_samples):
        """
        :param n_samples: How many plaintexts to draw
        :return: An array of plaintexts
        """
        # In most cases the format of the plain- and ciphertexts are the same,
        # so we can return random ciphertexts at this point
        return self.draw_ciphertexts(n_samples)

    def draw_ciphertexts(self, n_samples):
        """
        :param n_samples: How many ciphertexts to draw
        :return: An array of ciphertexts
        """
        bytes_per_word = self.bytes_per_word(self.word_size)
        ct = np.reshape(
            np.frombuffer(urandom(bytes_per_word * self.n_words * n_samples), dtype=self.word_dtype),
            (self.n_words, n_samples)
        )
        if self.word_size < 8:
            # Note: If the word size is greater than 8, it will always fit the dtype for the ciphers we use
            ct = np.right_shift(ct, 8 - self.word_size)
        return ct

    @abstractmethod
    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        pass

    def rol(self, x, k):
        """
        :param x: What to rotate
        :param k: How to rotate
        :return: x rotated by k bits to the left
        """
        return ((x << k) & self.mask_val) | (x >> (self.word_size - k))

    def ror(self, x, k):
        """
        :param x: What to rotate
        :param k: How to rotate
        :return: x rotated by k bits to the right
        """
        return (x >> k) | ((x << (self.word_size - k)) & self.mask_val)

    @staticmethod
    @abstractmethod
    def get_test_vectors():
        """
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        pass

    @classmethod
    def verify_test_vectors(cls):
        """
        Verifies the test vectors given by the designers
        :return: Result of the test
        """
        for cipher, pt, key, ct in cls.get_test_vectors():
            if not np.array_equal(ct, cipher.encrypt(pt, key)):
                print(f"ERROR: Test vector for {cls.__name__} not verified (encryption did not match)")
                return False
            try:  # This will use decrypt, which may not be implemented
                if not np.array_equal(pt, cipher.decrypt(ct, key)):
                    print(f"ERROR: Test vector for {cls.__name__} not verified (decryption did not match)")
                    return False
            except NotImplementedException as e:
                print(f"Info: Decryption not implemented for {cls.__name__}. (Original message: {e})")

        print(f"Info: All test vectors for {cls.__name__} verified")
        return True

## Clase del cipher

In [4]:
class Sist_Maria(AbstractCipher):

    def __init__(self, n_rounds=22, word_size=2, use_key_schedule=True, m=4):
        """
        Initializes a Speck-like cipher object
        """
        super(Sist_Maria, self).__init__(
            n_rounds, word_size, n_words=2,
            n_main_key_words=m, n_round_key_words=1,
            use_key_schedule=use_key_schedule
        )

        self.s_box = [0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD,
                      0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2]
        self.x_equivalency = [0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7,
                              0x8, 0x9, 0xA, 0xB, 0xC, 0xD, 0xE, 0xF]



    def bit_a_hexa(self, x):
        entero = int(x, 2)
        return hex(entero)

    def hexa_a_bit(self, x):
        entero = int(x, 16)
        return bin(entero)[2:].zfill(4)  # rellena con ceros a 4 bits


    def xor_bits(self, a, b):
      return ''.join(str(int(x) ^ int(y)) for x, y in zip(a, b))

    def encrypt_one_round(self, p, k, rc=None):
        x = self.bit_a_hexa(p)
        posicion = self.x_equivalency.index(int(x, 16))
        resultado_s_box = self.s_box[posicion]
        resultado = self.hexa_a_bit(hex(resultado_s_box))
        return resultado



    def decrypt_one_round(self, ciphertext, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        c0, c1 = c[0], c[1]
        c1 = c1 ^ c0
        c1 = self.ror(c1, self.beta)
        c0 = c0 ^ k[0]  # round key consists of one word only
        c0 = (c0 - c1) & self.mask_val
        c0 = self.rol(c0, self.alpha)
        return c0, c1

    def encrypt(self, plaintext, rounds, key=None):
        """
        Full encryption with trace: returns intermediate states for each round.
        :param plaintext: Tuple (left, right) as integers.
        :param key: Optional key for custom encryption (otherwise uses internal round keys).
        :return: List of tuples (L, R) representing the state after each round.
        """
        x, y = plaintext
        trace = [(x, y)]

        # Generate round keys if not provided
        if key is not None:
            round_keys = self.key_schedule(key)
        else:
            round_keys = self.round_keys  # Must be initialized before calling

        for r in range(rounds):
            x, y = self.encrypt_one_round((x, y), round_keys[r])
            trace.append((x, y))  # Save state after each round

        return trace

    def decrypt(self, ciphertext, rounds, key=None):
        """
        Full encryption with trace: returns intermediate states for each round.
        :param plaintext: Tuple (left, right) as integers.
        :param key: Optional key for custom encryption (otherwise uses internal round keys).
        :return: List of tuples (L, R) representing the state after each round.
        """
        x, y = ciphertext
        R = self.n_rounds if rounds is None else int(rounds)
        trace = [(x, y)]

        rk = self.key_schedule(key) if key is not None else self.round_keys
        rk = rk[:R]

        # Generate round keys if not provided
        if key is not None:
            round_keys = self.key_schedule(key)
        else:
            round_keys = self.round_keys  # Must be initialized before calling

        for r in range(R - 1, -1, -1):
            x, y = self.decrypt_one_round((x, y), rk[r])
            trace.append((x, y))
        return trace


    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        if variant != 1:
            raise Exception(f'ERROR: Variant {variant} of calculating back is not implemented')
        c0, c1 = c[0], c[1]
        c1 = c1 ^ c0
        c1 = self.ror(c1, self.beta)
        return c0, c1

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        ks = [0 for i in range(self.n_rounds)]
        ks[0] = key[len(key)-1]
        l = list(reversed(key[:len(key)-1]))

        for i in range(self.n_rounds-1):
            l[i % 3], ks[i+1] = self.encrypt_one_round((l[i % 3], ks[i]), [i])

        return np.array(ks, dtype=self.main_key_word_dtype)[:, np.newaxis]

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from https://eprint.iacr.org/2013/404.pdf, page 42
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        # Initialize Speck32/64 according to specification
        speck32_64 = Speck()
        key = np.array([[0x1918], [0x1110], [0x0908], [0x0100]], dtype=np.uint16)
        ks = speck32_64.key_schedule(key)
        pt = np.array([[0x6574], [0x694c]], dtype=np.uint16)
        ct = np.array([[0xa868], [0x42f2]], dtype=np.uint16)
        return [(speck32_64, pt, ks, ct)]


## Intanciar clase Maria y primera prueba

In [5]:
cipher = Sist_Maria()

In [6]:
plaintext = "1001"
cipher_text = cipher.encrypt_one_round(plaintext, 1)
print(cipher_text)

1110


## Encontrar la mejor mascara

### Convertir cadenas de bits en numeros

In [7]:
def to_int(x):
    """Convierte '1001', '0b1001', '0x9' o int a int."""
    if isinstance(x, int):
        return x
    s = str(x)
    if s.startswith("0b"):
        return int(s, 2)
    if s.startswith("0x"):
        return int(s, 16)
    # si sólo contiene 0/1 lo interpretamos como binario
    if all(ch in "01" for ch in s):
        return int(s, 2)
    # por compatibilidad, si es decimal en texto
    return int(s)

### De enteros a cadenas de bits

In [8]:
def to_bin_str(x, n_bits):
    """Formato binario con ceros a la izquierda: '101' -> '00000101' si n_bits=8"""
    return format(to_int(x), f"0{n_bits}b")

### Encontrar la paridad del entero

In [9]:
def parity_of_int(x: int) -> int:
    return bin(x).count("1") & 1

### Encontrar la mejor mascara

In [10]:
def linear_approximation_bias_from_pairs(
    pairs: list[tuple], mask_alpha, mask_beta, n_bits=None
) -> float:
    """
    Calcula el sesgo lineal |P[α·x = β·S(x)] - 0.5|

    Args:
        pairs: lista de tuplas (P, C) donde
               P = entrada a la S-box
               C = salida de la S-box
        mask_alpha: máscara de entrada (α)
        mask_beta:  máscara de salida (β)
        n_bits: tamaño de bloque en bits (opcional)

    Returns:
        Valor absoluto del sesgo respecto a 0.5
    """
    mA = to_int(mask_alpha)
    mB = to_int(mask_beta)
    N = len(pairs)
    if N == 0:
        return 0.0

    count_equal = 0
    for P, C in pairs:
        Pi = to_int(P)
        Ci = to_int(C)
        a_in = parity_of_int(Pi & mA)
        b_out = parity_of_int(Ci & mB)
        if a_in == b_out:
            count_equal += 1

    prob = count_equal / N
    return abs(prob - 0.5)

In [11]:
def find_best_masks_from_pairs(
    pairs, n_bits=4, candidate_masks_alpha=None, candidate_masks_beta=None, max_results=10
):
    """
    Busca las combinaciones de máscaras (α, β) con mayor sesgo lineal.

    Args:
        pairs: lista de tuplas (entrada, salida)
        n_bits: número de bits de la S-box
        candidate_masks_alpha: máscaras de entrada (por defecto todas ≠ 0)
        candidate_masks_beta: máscaras de salida (por defecto todas ≠ 0)
        max_results: número de resultados a retornar

    Returns:
        Lista de tuplas (α_bin, β_bin, bias) ordenadas por mayor bias
    """
    if candidate_masks_alpha is None:
        candidate_masks_alpha = list(range(1, 2**n_bits))
    else:
        candidate_masks_alpha = [to_int(m) for m in candidate_masks_alpha]

    if candidate_masks_beta is None:
        candidate_masks_beta = list(range(1, 2**n_bits))
    else:
        candidate_masks_beta = [to_int(m) for m in candidate_masks_beta]

    results = []
    for alpha in candidate_masks_alpha:
        for beta in candidate_masks_beta:
            bias = linear_approximation_bias_from_pairs(pairs, alpha, beta)
            results.append((alpha, beta, bias))

    # ordena por mayor sesgo
    results.sort(key=lambda x: x[2], reverse=True)

    return [
        (to_bin_str(alpha, n_bits), to_bin_str(beta, n_bits), bias)
        for (alpha, beta, bias) in results[:max_results]
    ]


In [12]:
def genrar_pares_toy_ex(numero_bits, cipher):
  pares = []
  for i in range(2**numero_bits):
    P = bin(i)[2:].zfill(numero_bits)
    C = cipher.encrypt_one_round(P, 1)
    print(f'P = {P}, C = {C}')
    pares.append((P, C))
  return pares


In [13]:
pares = genrar_pares_toy_ex(4, cipher)


P = 0000, C = 1100
P = 0001, C = 0101
P = 0010, C = 0110
P = 0011, C = 1011
P = 0100, C = 1001
P = 0101, C = 0000
P = 0110, C = 1010
P = 0111, C = 1101
P = 1000, C = 0011
P = 1001, C = 1110
P = 1010, C = 1111
P = 1011, C = 1000
P = 1100, C = 0100
P = 1101, C = 0111
P = 1110, C = 0001
P = 1111, C = 0010


In [14]:
mascaras = find_best_masks_from_pairs(pares)

In [15]:
mascaras

[('0001', '0101', 0.25),
 ('0001', '0111', 0.25),
 ('0001', '1101', 0.25),
 ('0001', '1111', 0.25),
 ('0010', '1011', 0.25),
 ('0010', '1101', 0.25),
 ('0011', '0110', 0.25),
 ('0011', '1010', 0.25),
 ('0100', '0111', 0.25),
 ('0100', '1011', 0.25)]

In [16]:
r1 = find_best_masks_from_pairs(pares)
r2 = []

for mP in range(1, 16):
    for mC in range(1, 16):
        bias = linear_approximation_bias_from_pairs(pares, mC, mP)  # invertidos
        if bias > 0:
            r2.append((to_bin_str(mP, 4), to_bin_str(mC, 4), bias))

r2.sort(key=lambda x: x[2], reverse=True)
print(r1[:5])
print(r2[:5])


[('0001', '0101', 0.25), ('0001', '0111', 0.25), ('0001', '1101', 0.25), ('0001', '1111', 0.25), ('0010', '1011', 0.25)]
[('0001', '1001', 0.25), ('0001', '1011', 0.25), ('0001', '1101', 0.25), ('0001', '1111', 0.25), ('0010', '1010', 0.25)]


In [17]:
pip install cryptanalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 35.6 MB/s eta 0:00:00


In [18]:
import cryptanalysis
from cryptanalysis.utils import calculate_linear_bias
from collections import Counter

# Ejemplo S-box (PRESENT 4-bit)
S = [0xC,0x5,0x6,0xB,0x9,0x0,0xA,0xD,0x3,0xE,0xF,0x8,0x4,0x7,0x1,0x2]

biases = calculate_linear_bias(S, no_sign=True, fraction=True)  # fraction=True -> bias como fracción
# biases es un Counter con claves (imask, omask) -> bias (abs or fraction según flags)

# ordenar y mostrar top
top = sorted(biases.items(), key=lambda kv: abs(kv[1]), reverse=True)[:10]
for (imask, omask), b in top:
    print(f"in_mask={imask:04b}, out_mask={omask:04b}, bias={b}")


calculating sbox bias: 100%|██████████| 16/16 [00:00<00:00, 4950.13it/s]

in_mask=0000, out_mask=0000, bias=0.5
in_mask=0001, out_mask=0101, bias=0.25
in_mask=0001, out_mask=0111, bias=0.25
in_mask=0001, out_mask=1101, bias=0.25
in_mask=0001, out_mask=1111, bias=0.25
in_mask=0010, out_mask=1011, bias=0.25
in_mask=0010, out_mask=1101, bias=0.25
in_mask=0011, out_mask=0110, bias=0.25
in_mask=0011, out_mask=1010, bias=0.25
in_mask=0100, out_mask=0111, bias=0.25


## Prueba Toy Example

In [19]:
def parity_array(x):
    # vectorized paridad (0/1) para array de enteros
    # usa bit_count (Python 3.8+) o bin(x).count('1')
    return np.array([int(n.bit_count() & 1) for n in x], dtype=np.uint8)

def empirical_bias(plain_arr, cipher_arr, mask_P, mask_C):
    """
    plain_arr, cipher_arr: numpy arrays (dtype=int) de la misma longitud N
    mask_P, mask_C: enteros (máscaras)
    devuelve: (p, epsilon, count_equal, N)
      p = freq de igualdad = count_equal / N
      epsilon = p - 0.5
    """
    N = len(plain_arr)
    assert len(cipher_arr) == N
    pbits = parity_array(plain_arr & mask_P)
    cbits = parity_array(cipher_arr & mask_C)
    equal = (pbits == cbits).sum()
    p = equal / N
    eps = p - 0.5
    return p, eps, int(equal), N

def z_score_from_bias(eps, N):
    """
    Z = observed_deviation / std_error
    std_error ~ 0.5 / sqrt(N)  (since p≈0.5 => var ≈ 0.25)
    => Z = eps / (0.5 / sqrt(N)) = 2 * eps * sqrt(N)
    """
    if N <= 0: return 0.0
    return 2.0 * eps * sqrt(N)

def required_N_for_bias(eps, z_target=3.29):
    """
    Aproximación para N necesaria para detectar bias eps con z >= z_target.
    z_target por ejemplo: 1.96 (alpha=0.05), 2.575 (alpha=0.01), 3.29 (alpha=0.001)
    Fórmula: N = (z / (2*eps))^2
    """
    if abs(eps) == 0:
        return float('inf')
    return (z_target / (2.0 * abs(eps)))**2

## Sist Maria para el ataque

In [20]:
class Sist_Maria_Ataque(AbstractCipher):
    """
    SPN simple de 16 bits (4 nibbles) con S-box 4x4 para facilitar
    criptoanálisis lineal (Matsui 1).
    """

    def __init__(self, n_rounds=10, word_size=16, use_key_schedule=True, m=4, master_key=None):
        """
        n_rounds: rondas del SPN (>= 4 recomendado)
        word_size: bits del bloque (16 en este diseño)
        m: número de palabras en la llave maestra (cada una de 16 bits) si usas key_schedule tipo "toy"
        master_key: np.array shape (m,1) de uint16 o lista de ints (opcional)
        """
        # NOTA: para este SPN el bloque es UN solo word de 16 bits, así que n_words=1
        super(Sist_Maria_Ataque, self).__init__(
            n_rounds, word_size, n_words=1,
            n_main_key_words=m, n_round_key_words=1,
            use_key_schedule=use_key_schedule
        )

        # --- S-box 4x4 e inversa ---
        self.s_box = [0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD,
                      0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2]
        self.s_box_inv = [0]*16
        for i, v in enumerate(self.s_box):
            self.s_box_inv[v] = i

        # --- P-box como permutación de nibbles (LSB nibble = índice 0) ---
        # Ejemplo: [x3, x2, x1, x0]  -> [x1, x3, x0, x2]
        self.P = [1, 3, 0, 2]
        self.P_inv = [0]*4
        for old_pos, new_pos in enumerate(self.P):
            self.P_inv[new_pos] = old_pos

        # Máscaras
        self.block_bits = 16
        self.mask_val = (1 << self.block_bits) - 1

        # Round keys (R+1 si usas subclave final)
        self.R = int(n_rounds)
        if master_key is not None:
            self.round_keys = self.key_schedule(master_key)
        else:
            # si AbstractCipher ya te arma self.main_key, úsala:
            # (asegúrate de que sea consistente con key_schedule)
            self.round_keys = None  # se inicializa al primer encrypt si es necesario

    # ================== Helpers de nibbles ==================
    @staticmethod
    def _get_nibble(x, j):
        return (x >> (4*j)) & 0xF

    @staticmethod
    def _set_nibble(x, j, v):
        mask = ~(0xF << (4*j)) & 0xFFFF
        return (x & mask) | ((v & 0xF) << (4*j))

    def _sbox_layer(self, x):
        y = 0
        for j in range(4):
            nib = self._get_nibble(x, j)
            y = self._set_nibble(y, j, self.s_box[nib])
        return y

    def _sbox_inv_layer(self, x):
        y = 0
        for j in range(4):
            nib = self._get_nibble(x, j)
            y = self._set_nibble(y, j, self.s_box_inv[nib])
        return y

    def _pbox_layer(self, x):
        n = [self._get_nibble(x, j) for j in range(4)]
        n2 = [0]*4
        for old_pos, new_pos in enumerate(self.P):
            n2[new_pos] = n[old_pos]
        y = 0
        for j, val in enumerate(n2):
            y = self._set_nibble(y, j, val)
        return y

    def _pbox_inv_layer(self, x):
        n = [self._get_nibble(x, j) for j in range(4)]
        n2 = [0]*4
        for new_pos, old_pos in enumerate(self.P_inv):
            n2[old_pos] = n[new_pos]
        y = 0
        for j, val in enumerate(n2):
            y = self._set_nibble(y, j, val)
        return y

    # ================== Ronda ==================
    def encrypt_one_round(self, x, k, last_round=False):
        """
        x: entero de 16 bits (estado)
        k: entero de 16 bits (subclave de ronda)
        last_round: si True, no aplica P-box
        Orden: XOR subclave -> S-box -> (P-box si no es última)
        """
        x = (x ^ k) & self.mask_val
        x = self._sbox_layer(x)
        if not last_round:
            x = self._pbox_layer(x)
        return x

    def decrypt_one_round(self, x, k, last_round=False):
        """
        Inversa de la ronda anterior.
        Orden inverso: (P⁻¹ si no es última) -> S⁻¹ -> XOR subclave
        """
        if not last_round:
            x = self._pbox_inv_layer(x)
        x = self._sbox_inv_layer(x)
        x = (x ^ k) & self.mask_val
        return x

    # ================== Cifrado/Descifrado ==================
    def encrypt(self, plaintext, rounds=None, key=None):
        """
        plaintext: entero de 16 bits (no tupla)
        key: opcional. Si se da, invoca key_schedule(key).
        Devuelve entero de 16 bits (ciphertext).
        """
        R = self.R if rounds is None else int(rounds)
        if key is not None:
            rk = self.key_schedule(key)
        else:
            rk = self.round_keys
            if rk is None:
                # intenta derivar desde self.main_key si AbstractCipher la trae
                base_key = key if key is not None else getattr(self, "main_key", None)
                if base_key is None:
                    raise ValueError("No hay master_key para derivar round_keys.")
                rk = self.key_schedule(base_key)

        x = int(plaintext) & self.mask_val
        # Rondas 0..R-2 con P-box
        for r in range(R-1):
            x = self.encrypt_one_round(x, int(rk[r, 0]), last_round=False)
        # Última ronda sin P-box + subclave final opcional
        x = self.encrypt_one_round(x, int(rk[R-1, 0]), last_round=True)
        # Subclave final (opcional): si quieres, agrega una más:
        if rk.shape[0] > R:
            x ^= int(rk[R, 0])
            x &= self.mask_val
        return x

    def decrypt(self, ciphertext, rounds=None, key=None):
        R = self.R if rounds is None else int(rounds)
        if key is not None:
            rk = self.key_schedule(key)
        else:
            rk = self.round_keys
            if rk is None:
                base_key = key if key is not None else getattr(self, "main_key", None)
                if base_key is None:
                    raise ValueError("No hay master_key para derivar round_keys.")
                rk = self.key_schedule(base_key)

        x = int(ciphertext) & self.mask_val
        # Si hay subclave final extra, quítala primero
        if rk.shape[0] > R:
            x ^= int(rk[R, 0])
            x &= self.mask_val
        # Última ronda (inversa) sin P-box
        x = self.decrypt_one_round(x, int(rk[R-1, 0]), last_round=True)
        # Rondas R-2..0 (inversas) con P-box inversa
        for r in range(R-2, -1, -1):
            x = self.decrypt_one_round(x, int(rk[r, 0]), last_round=False)
        return x

    def calc_back(self, c, p=None, variant=1):
        """
        'Deshace' lo determinístico del final para análisis:
        quita subclave final (si existe) y S-box final (sin P-box).
        Devuelve el estado antes de la última S-box, útil en ataques.
        """
        if variant == 0:
            return c
        if variant != 1:
            raise Exception(f'ERROR: Variant {variant} no implementada')
        x = int(c) & self.mask_val
        # quita subclave final opcional
        if self.round_keys is not None and self.round_keys.shape[0] > self.R:
            x ^= int(self.round_keys[self.R, 0])
            x &= self.mask_val
        # quita última ronda S-box (sin P-box) y XOR de subclave
        x = self._sbox_inv_layer(x)
        if self.round_keys is None:
            raise ValueError("round_keys no inicializadas.")
        x ^= int(self.round_keys[self.R-1, 0])
        x &= self.mask_val
        return x

    # ================== Key schedule ==================
    def key_schedule(self, key):
        """
        key puede ser:
        - np.array shape (m,1) dtype=uint16   (estilo AbstractCipher)
        - lista/tupla de ints de 16 bits
        Genera R (+1 opcional) subclaves de 16 bits.
        """
        import numpy as np

        # Normaliza a lista de ints
        if isinstance(key, np.ndarray):
            base = [int(x) & 0xFFFF for x in key.flatten().tolist()]
        elif isinstance(key, (list, tuple)):
            base = [int(x) & 0xFFFF for x in key]
        else:
            # si dan un int, lo expandimos en m palabras "toy"
            base = [(int(key) >> (16*i)) & 0xFFFF for i in range(max(1, self.n_main_key_words))]

        m = max(1, self.n_main_key_words)
        if len(base) < m:
            # rellena si es necesario
            base = (base + [0]*m)[:m]

        R = self.R
        rks = []
        state = base[:]  # lista de m palabras de 16 bits

        # key schedule "toy": rotaciones y XOR con constantes de ronda
        for r in range(R + 1):  # +1 para subclave final opcional
            # mezcla ligera
            w = ((state[0] << 5) | (state[0] >> 11)) & 0xFFFF
            w ^= (0x9E37 ^ r) & 0xFFFF
            # rota vector y mete w
            state = state[1:] + [w]
            rk = state[-1]  # toma la última como subclave
            rks.append(rk & 0xFFFF)

        import numpy as np
        self.round_keys = np.array(rks, dtype=np.uint16)[:, np.newaxis]
        return self.round_keys

    # Opcional: test vectors propios (aquí vacío para no confundir con Speck)
    @staticmethod
    def get_test_vectors():
        return []

## Ataque Matsui 1

In [21]:
alfa = "0001"
beta = "0101"

alpha_4 = int(alfa, 2)
beta_4  = int(beta, 2)

In [22]:
alpha_4

1

In [23]:
beta_4

5

In [24]:
def parity_of_int(x: int) -> int:
    # paridad XOR de todos los bits en 1
    return bin(x).count("1") & 1

### Linear Approximation Table

In [25]:
def build_LAT(sbox):
    # Tabla de aproximaciones lineales de la S-box 4x4
    LAT = [[0]*16 for _ in range(16)]
    for a in range(16):
        for b in range(16):
            cnt = 0
            for x in range(16):
                if parity_of_int(a & x) == parity_of_int(b & sbox[x]):
                    cnt += 1
            LAT[a][b] = cnt - 8  # = 16 * ε
    return LAT

In [26]:

def best_sbox_masks(sbox, exclude_trivial=True, topk=10):
    LAT = build_LAT(sbox)
    cand = []
    for a in range(16):
        for b in range(16):
            if exclude_trivial and (a == 0 or b == 0):
                continue
            eps = LAT[a][b]/16.0
            cand.append((abs(eps), eps, a, b))
    cand.sort(reverse=True, key=lambda t: t[0])
    return cand[:topk]

In [27]:
def required_N_for_bias(eps, z_target=3.29):
    """
    Tamaño muestral aproximado para detectar un sesgo |eps|
    con significancia ~ z_target (3.29 ~ p≈0.001).
    Modelo normal: z = (count - N/2) / (sqrt(N)/2) ≈ 2*eps*sqrt(N)
    => N ≈ (z/(2*eps))^2
    """
    if eps <= 0:
        raise ValueError("eps debe ser positivo")
    return ceil((z_target / (2*eps))**2)

In [28]:
def gen_pairs(cipher, N, rng=None):
    """
    Genera N pares (P,C) con la misma key interna de 'cipher'.
    'cipher.encrypt' debe aceptar un entero 16b y devolver entero 16b.
    """
    if rng is None:
        rng = random
    P = []
    C = []
    for _ in range(N):
        p = rng.getrandbits(16)
        c = cipher.encrypt(p)  # usa round_keys ya fijadas
        P.append(p)
        C.append(c)
    return P, C


In [29]:
def matsui1_nibble_attack(cipher, P, C, alpha_mask_int, beta_mask_4bit, j, xor_first=True):
    """
    Ataca un nibble j (0=LSB) de la última ronda.
    xor_first=True si la última ronda es XOR(subclave)->Sbox (lo usual aquí).
    Si tu última ronda fuera S->XOR, usa xor_first=False (ver línea marcada).
    Devuelve lista ordenada [(|eps|, eps, k_guess, aciertos, N), ...]
    """
    N = len(P)
    results = []
    for k_guess in range(16):
        equal = 0
        for p, c in zip(P, C):
            Cj = (c >> (4*j)) & 0xF
            if xor_first:
                # Última ronda: x ^= k ; x = S(x)
                Uj = cipher.s_box_inv[Cj ^ k_guess]
            else:
                # Si fuera S->XOR: x = S(x) ; x ^= k
                Uj = cipher.s_box_inv[Cj] ^ k_guess
            pin  = parity_of_int(p & alpha_mask_int)
            uout = parity_of_int(Uj & beta_mask_4bit)
            if (pin ^ uout) == 0:
                equal += 1
        eps_hat = (equal / N) - 0.5
        results.append((abs(eps_hat), eps_hat, k_guess, equal, N))
    results.sort(reverse=True, key=lambda t: t[0])
    return results

In [30]:
def matsui1_full_last_round_key(cipher, P, C, alpha_mask_int, beta_mask_4bit, xor_first=True):
    """
    Ejecuta el ataque para j=0..3 y devuelve:
    - best_key_16: entero 16b de la subclave final estimada
    - detail: dict j -> top lista de candidatos
    """
    detail = {}
    key_nibbles = [0]*4
    for j in range(4):
        res = matsui1_nibble_attack(cipher, P, C, alpha_mask_int, beta_mask_4bit, j, xor_first=xor_first)
        detail[j] = res
        key_nibbles[j] = res[0][2]  # k_guess con mayor |eps|
    # Empaquetar nibbles en 16 bits: j=0 es LSB nibble
    best_key_16 = 0
    for j in range(4):
        best_key_16 |= (key_nibbles[j] & 0xF) << (4*j)
    return best_key_16, detail

In [31]:
def run_matsui1_demo(cipher, N=None, alpha_mask_int=None, beta_mask_4bit=None, j_demo=0, rng=None):
    """
    1) Selecciona (α,β) si no se dieron
    2) Estima N si no se dio
    3) Genera pares
    4) Ataca nibble demo y la subclave completa
    """
    # 1) Elegir (α,β) fuerte desde LAT si no se dan
    if alpha_mask_int is None or beta_mask_4bit is None:
        top = best_sbox_masks(cipher.s_box, exclude_trivial=True, topk=1)
        _, eps_theory, a4, b4 = top[0]
        # α en este ejemplo la ponemos sobre el nibble 0 del plaintext:
        alpha_mask_int = a4  # si quieres activar otro nibble, desplaza a la posición deseada
        beta_mask_4bit = b4

    # 2) Estimar N si no se da (usa eps teórico como guía)
    if N is None:
        eps_used = max(1e-6, abs(eps_theory))  # protección
        N = max(2000, required_N_for_bias(eps_used, z_target=3.29))

    # 3) Generar pares
    P, C = gen_pairs(cipher, N, rng=rng)

    # 4) Atacar un nibble demo
    res_demo = matsui1_nibble_attack(cipher, P, C, alpha_mask_int, beta_mask_4bit, j_demo, xor_first=True)
    best_demo = res_demo[0]

    # 5) Atacar los 4 nibbles (subclave final completa)
    k_last, detail = matsui1_full_last_round_key(cipher, P, C, alpha_mask_int, beta_mask_4bit, xor_first=True)

    return {
        "alpha_mask_int": alpha_mask_int,
        "beta_mask_4bit": beta_mask_4bit,
        "N": N,
        "demo_j": j_demo,
        "demo_top": res_demo[:4],
        "k_last_round_est": k_last,
        "detail": detail
    }

In [32]:
master_key = np.array([[0x1234],[0xABCD],[0x0F0F],[0xFEDC],], dtype=np.uint16)
cipher = Sist_Maria_Ataque(n_rounds=10, word_size=16, use_key_schedule=True, m=4, master_key=master_key)

In [33]:
rng = random.Random(123)

In [34]:
out = run_matsui1_demo(cipher, N=8000, alpha_mask_int=alpha_4, beta_mask_4bit=beta_4, j_demo=0, rng=rng)

### Resultados

In [35]:
print("α (plaintext mask) =", hex(out["alpha_mask_int"]))
print("β (U_j mask)       =", bin(out["beta_mask_4bit"]))
print("N usados            =", out["N"])
print("Nibble demo j       =", out["demo_j"])
print("Top 4 k_j candidatos (|eps|, eps, k, aciertos, N):")

α (plaintext mask) = 0x1
β (U_j mask)       = 0b101
N usados            = 8000
Nibble demo j       = 0
Top 4 k_j candidatos (|eps|, eps, k, aciertos, N):


In [36]:
for t in out["demo_top"]:
    print("  ", t)
print("Subclave final estimada (16b): 0x%04X" % out["k_last_round_est"])

   (0.01200000000000001, 0.01200000000000001, 3, 4096, 8000)
   (0.01200000000000001, -0.01200000000000001, 6, 3904, 8000)
   (0.011124999999999996, -0.011124999999999996, 4, 3911, 8000)
   (0.010875000000000024, -0.010875000000000024, 11, 3913, 8000)
Subclave final estimada (16b): 0x3D53


In [37]:
rng = np.random.default_rng(12345)

def make_one_round_pairs(cipher, N=1<<14, include_pbox=True, master_key=None):
    """
    Genera pares (P, C1) donde C1 = salida de *una sola ronda*.
    - include_pbox=True -> aplica P-box (ronda intermedia)
    - include_pbox=False -> sin P-box (como última ronda)
    """
    # Asegura round keys
    if cipher.round_keys is None:
        if master_key is None:
            master_key = [0x1234, 0x5678, 0x9ABC, 0xDEF0]  # cualquiera
        cipher.key_schedule(master_key)

    k0 = int(cipher.round_keys[0,0])
    pairs = []
    for _ in range(N):
        P = rng.integers(0, 1<<cipher.block_bits, dtype=np.uint16).item()
        C1 = cipher.encrypt_one_round(P, k0, last_round=not include_pbox)
        pairs.append((P, C1))
    return pairs

def bias_one_round(cipher, alpha_mask, beta_mask, N=1<<16, include_pbox=True):
    pairs = make_one_round_pairs(cipher, N=N, include_pbox=include_pbox)
    return linear_approximation_bias_from_pairs(
        pairs, mask_alpha=alpha_mask, mask_beta=beta_mask
    )

In [38]:
def permute_mask_nibbles(mask16, P):
    """Mueve nibbles de mask16 según P (lista como [1,3,0,2])."""
    out = 0
    for old_pos, new_pos in enumerate(P):
        nib = (mask16 >> (4*old_pos)) & 0xF
        out |= (nib << (4*new_pos))
    return out


In [39]:
cipher = Sist_Maria_Ataque(n_rounds=10, word_size=16, m=4)
alpha = 0x000F        # nibble 0
beta  = 0x00F0        # nibble 1 (después de P-box)
b = bias_one_round(cipher, alpha, beta, N=1<<16, include_pbox=True)
print("Sesgo 1 ronda (con P-box):", b)


Sesgo 1 ronda (con P-box): 0.0006103515625
